# Exploratory Data Analysis

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
import os

np.random.seed(1320210409)
randomstate = np.random.RandomState(1320210409)

# The data

## Features

All features are hourly and a country-wide average.
- **Time** _[YYYY-MM-DD HH:MM:SS]_
- **el_load:** electricity load _[MW]_
- **prec:** rainfall amount _[mm]_
- **temp:** temperature _[°C]_
- **rhum:** relative humidity [%]
- **grad:** global radiation _[J/cm²]_
- **pres:** momentary sea level air pressure _[hPa]_
- **wind:** average wind speed _[m/s]_
- **Vel_tviz:** Velence water temperature in Agárd _[°C]_
- **Bal_tviz:** Balaton water temperature in Siófok _[°C]_
- **holiday:** 1 or 0 depending on if it's a holiday
- **weekend:** 1 or 0 depending on if it's a weekend
- **covid:** 1 or 0 depending on covid restrictions in Hungary (estimate)

### The goal

I want to predict Hungary's electricity load for the **next couple of hours** using this dataset, or it's differently aggregated counterpart (country, region, county or station)

In [ ]:
df = pd.read_csv(
    'data/final_dataframe.csv',
    parse_dates=['Time'],
    index_col='Time',
    sep=';'
)

df.info()

df

No null entries, I have dealt with those in the _data_organization_ notebook.

In [ ]:
df['hour'] = df.index.hour
df['weekday'] = df.index.weekday
df['dayofmonth'] = df.index.day
df['dayofyear'] = df.index.dayofyear
df['month'] = df.index.month
df['year'] = df.index.year

df

## Features

- **Time** _[YYYY-MM-DD HH:MM:SS]_
- **el_load:** electricity load _[MW]_
- **prec:** rainfall amount _[mm]_
- **temp:** temperature _[°C]_
- **rhum:** relative humidity [%]
- **grad:** global radiation _[J/cm²]_
- **pres:** momentary sea level air pressure _[hPa]_
- **wind:** average wind speed _[m/s]_
- **Vel_tviz:** Velence water temperature in Agárd _[°C]_
- **Bal_tviz:** Balaton water temperature in Siófok _[°C]_
- **holiday:** 1 or 0 depending on if it's a holiday
- **weekend:** 1 or 0 depending on if it's a weekend
- **covid:** 1 or 0 depending on covid restrictions in Hungary (estimate)

In [ ]:
group_by = ['hour', 'weekday', 'dayofmonth', 'dayofyear', 'month', 'year']

def plot_feature(dataframe: pd.DataFrame, groupes: list, feature: str, desc: str, color: str):
    group_len = len(groupes)
    fig, ax = plt.subplots(2, group_len // 2, figsize=(20, 7))
    fig.suptitle(f"Feature: {feature} ({desc})")
    for i, ax in enumerate(ax.flatten()):
        group = groupes[i % group_len]
        grouped = dataframe.groupby(group)[feature].mean()
        ax.set_title(f"Grouped by {group}", fontsize=10)
        marker = 'o' if group != 'dayofyear' else None
        ax.plot(grouped, color=color, marker=marker)

#### Electricity load

In [ ]:
print(df['el_load'].describe())
print(f"0.1st percentile {df['el_load'].quantile(0.001)}")
print(f"99.9th percentile {df['el_load'].quantile(0.999)}")

plot_feature(df, group_by, 'el_load', 'Electricity load', 'black')

- the description tells us, that the standard deviation is very high for the elecrictiy load, which is to be expected
- min and max values suggest outliers, which i'll remove by adjusting them to above seen percintiles
- daily average rises during the day, it hits its peak at 18-19
- lower during the weekend
- we don't learn too much from the day of the month at this time
- during the year, load is higher in winter, probably since there's less sunlight
- we can see the effects of covid between 2020-2022

In [ ]:
df['el_load'] = df['el_load'].clip(
    lower=df['el_load'].quantile(0.001),
    upper=df['el_load'].quantile(0.999)
)

df['el_load'].describe()

#### Precipitation

In [ ]:
print(df['prec'].describe())
print(f"0.1st percentile {df['prec'].quantile(0.001)}")
print(f"99.9th percentile {df['prec'].quantile(0.999)}")

plot_feature(df, group_by, 'prec', 'Precipitation', 'blue')

- precipitation is higher during the summer as expected
- it's higher during the weekend, but that's probably up to chance
- it's higher during the afternoon and evening
- other groups tell us nothing

#### Temperature

In [ ]:
print(df['temp'].describe())
print(f"0.1st percentile {df['temp'].quantile(0.001)}")
print(f"99.9th percentile {df['temp'].quantile(0.999)}")

plot_feature(df, group_by, 'temp', 'Temperature', 'red')

- temperature is higher during the summer as expected
- it's also higher during the day as expected
- seemingly, it's rising slowly as the years go on with some outliers

#### Relative humidity

In [ ]:
print(df['rhum'].describe())
print(f"0.1st percentile {df['rhum'].quantile(0.001)}")
print(f"99.9th percentile {df['rhum'].quantile(0.999)}")

plot_feature(df, group_by, 'rhum', 'Relative humidity', 'green')

- humidity is lower during the day, hitting its low in the afternoon
- the week group is a decieving graph, since the values are so close to each other
- it's lower during the summer overall

#### Global radiation

In [ ]:
print(df['grad'].describe())
print(f"0.1st percentile {df['grad'].quantile(0.001)}")
print(f"99.9th percentile {df['grad'].quantile(0.999)}")

plot_feature(df, group_by, 'grad', 'Global radiation', 'orange')

- global radiation is higher during the summer and the day as expected
- it's slowly increasing as the years go on, 2023 being an outlier since we only have data for 8 months there

#### Momentary sea level air pressure

In [ ]:
print(df['pres'].describe())
print(f"0.1st percentile {df['pres'].quantile(0.001)}")
print(f"99.9th percentile {df['pres'].quantile(0.999)}")

plot_feature(df, group_by, 'pres', 'Momentary sea level air pressure', 'purple')

- air pressure fluctuates heavily during the day, being higher in the morning, but low in the afternoon
- it's higher during the winter
- other groups tell us nothing

#### Average wind speed

In [ ]:
print(df['wind'].describe())
print(f"0.1st percentile {df['wind'].quantile(0.001)}")
print(f"99.9th percentile {df['wind'].quantile(0.999)}")

plot_feature(df, group_by, 'wind', 'Average wind speed', 'brown')

- wind speed is higher during the day, hitting its peak in the afternoon
- it's higher during the winter and spring

#### Water temperature for Balaton and Velence

In [ ]:
print(df['Vel_tviz'].describe())
print(f"0.1st percentile {df['Vel_tviz'].quantile(0.001)}")
print(f"99.9th percentile {df['Vel_tviz'].quantile(0.999)}")

plot_feature(df, group_by, 'Vel_tviz', 'Velence water temperature in Agárd', 'cyan')

print(df['Bal_tviz'].describe())
print(f"0.1st percentile {df['Bal_tviz'].quantile(0.001)}")
print(f"99.9th percentile {df['Bal_tviz'].quantile(0.999)}")

plot_feature(df, group_by, 'Bal_tviz', 'Balaton water temperature in Siófok', 'lightblue')

- the 2 water temperature graphs are really similar, so I'll write about them together
- water temperature is higher during the summer as expected
- they hit their peak in the afternoon for obvious reasons

## Correletion matrix

In [ ]:
# limit features used
corr = df.drop(columns=['holiday', 'weekend', 'covid']).corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corr, annot=True, cmap='coolwarm')

- tempretaure and water temperatures are highly correlated as expected
- the 2 water temperatures correlate highly, but I will keep these features seperate for now
- dayofyear and month are highly correlated, but that's to be expected
- relative humidity and global radiation display inverse correlation, which is interesting

## Lag features

In time-series problems, it's generally a good practice to use lag features on the datapoint we try to predict, I'll add 4 different lags of el_load to the dataset.

In [ ]:
df['el_load_lag24'] = df['el_load'].shift(24, fill_value=0)
df['el_load_lag48'] = df['el_load'].shift(48, fill_value=0)
df['el_load_lag72'] = df['el_load'].shift(72, fill_value=0)
df['el_load_lag96'] = df['el_load'].shift(96, fill_value=0)

df

## Automatic feature selection

I'll be doing a 3 hour forecast, using the last 24 hours, to get the best features I can.

In [ ]:
X = df.to_numpy(dtype=np.float64)
y = df['el_load'].to_numpy(dtype=np.float64)
rf_X = np.zeros((len(X)-24-3, 24*X.shape[1]))
rf_y = np.zeros((len(X)-24-3, 3))
for i in range(len(X)-24-3):
    rf_X[i] = X[i:i+24].flatten()
    rf_y[i] = y[i+24:i+24+3]

# Feature groups exist, to simulate choosing or not choosing a feature/column rather than evaluating every possible combination
    # for example, I won't check if the 2nd or 3rd hour of the day is a good feature, but rather if the hour of the day is a good feature
feature_groups = [[j for j in range(i, rf_X.shape[1], len(df.columns))] for i in range(len(df.columns))]

start_from_scratch = False
# init, option to load already computed results
fixed_features = None
scores_df = pd.DataFrame(columns=['score'] + list(df.columns))
range_start = 1
if not start_from_scratch and os.path.exists('data/feature_selection.csv'):
    scores_df = pd.read_csv('data/feature_selection.csv', sep=';', index_col=0)
    range_start = len(scores_df.index) + 1
    last_row = tuple(scores_df.iloc[-1][1:].to_numpy(dtype=bool))
    fixed_features = [feat for feat, use in zip(feature_groups, last_row) if use]
    fixed_features = tuple([item for sublist in fixed_features for item in sublist])
    

for i in range(range_start, len(df.columns)+1):
    ts_split = TimeSeriesSplit(n_splits=4).split(rf_X)
    sfs = SFS(
        RandomForestRegressor(n_jobs=-1, random_state=randomstate, n_estimators=40,
                              max_depth=20, max_features=1.0),
        k_features=i,
        forward=True,
        scoring='neg_root_mean_squared_error', # regular root mean squared error doesn't exist in sklearn
        cv = list(ts_split),
        fixed_features=fixed_features,
        feature_groups=feature_groups
    )

    sfs.fit_transform(rf_X, rf_y)
    fixed_features = tuple(sfs.k_feature_idx_)
    feature_idxs = [True if i in fixed_features else False for i in range(len(df.columns))]
    scores_df.loc[i] = [-sfs.k_score_] + feature_idxs
    print(f"Finished {i} feature(s), score: {-sfs.k_score_}")
    scores_df.to_csv('data/feature_selection.csv', sep=';')

scores_df

It hits the best performance at 11 features.

## How much time should I use for the forecast?

In [ ]:
def score_of_lookback(X: np.ndarray, y: np.ndarray,seq_len: int, pred_len: int):
    rf_X = np.zeros((len(X)-seq_len-pred_len, seq_len*X.shape[1]))
    rf_y = np.zeros((len(X)-seq_len-pred_len, pred_len))
    for i in range(len(X)-seq_len-pred_len):
        rf_X[i] = X[i:i+seq_len].flatten()
        rf_y[i] = y[i+seq_len:i+seq_len+pred_len]

    ts_split = TimeSeriesSplit(n_splits=4).split(rf_X)
    
    model = RandomForestRegressor(n_estimators=75, n_jobs=-1, random_state=randomstate)
    scores = cross_val_score(model, rf_X, rf_y, scoring='neg_root_mean_squared_error', cv=list(ts_split))
    
    print(f"Score for {seq_len} hours of data: {-scores.mean()}, std: {scores.std()}")
    return -scores.mean(), scores.std()

new_df = df[['el_load', 'prec', 'grad', 'holiday', 'weekend', 'hour', 'weekday', 'dayofyear', 'month', 'year', 'el_load_lag24']]

X = new_df.to_numpy(dtype=np.float64)
y = new_df['el_load'].to_numpy(dtype=np.float64)

score_of_lookback(X, y, 12, 3)
score_of_lookback(X, y, 24, 3)
score_of_lookback(X, y, 36, 3)
score_of_lookback(X, y, 48, 3)

Looks like 24 hours is the best option.